# 2. Beta–Binomial Classifier

Load grouped count data, compute posterior summaries, and assign mastery labels.


In [1]:
import pandas as pd
from pathlib import Path
from scipy.stats import beta

alpha_prior, beta_prior = 2.0, 2.0
familiar, proficient, confidence = 0.3, 0.5, 0.8

examples = pd.read_csv(Path('../data/beta_binomial_examples.csv'))

rows = []
for _, r in examples.iterrows():
    a_post = alpha_prior + int(r.successes)
    b_post = beta_prior + (int(r.attempts) - int(r.successes))
    mean = a_post / (a_post + b_post)
    ci_low = beta.ppf((1-confidence)/2, a_post, b_post)
    label = 'Proficient' if ci_low >= proficient else ('Familiar' if ci_low >= familiar else 'Attempted')
    rows.append({**r.to_dict(), 'post_mean': mean, 'ci_low': ci_low, 'label': label})

pd.DataFrame(rows)


,group_id,attempts,successes,post_mean,ci_low,label
0,A,2,1,0.500000,0.246636,Attempted
1,A,5,3,0.555556,0.344623,Familiar
2,B,1,0,0.400000,0.142559,Attempted
3,B,7,6,0.727273,0.550396,Proficient
4,C,10,4,0.428571,0.263730,Attempted
5,D,3,3,0.714286,0.489684,Familiar
6,E,8,2,0.333333,0.169233,Attempted
7,F,12,9,0.687500,0.536029,Proficient
8,G,4,1,0.375000,0.169638,Attempted
9,H,6,5,0.700000,0.509919,Proficient
